In [1]:
import pandas as pd
import math
import string
import re
from myFunctions import processSentences, meanVector, cosineDistance
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statistics




# NLP Packages
from gensim.models.phrases import Phrases, Phraser
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
reddit = pd.read_csv("data/reddit_wsb.csv")
reddit2 = pd.read_csv("data/r_wallstreetbets_posts.csv")

sentences_raw = []


for i in range(len(reddit)):
    body = reddit['body'][i]
    if isinstance(body, str):
        sentences_raw.extend(body.split('.'))

for i in range(len(reddit2)):
    title = reddit2['title'][i]
    if isinstance(title, str):
        sentences_raw.append(title)
        
        

text8 = api.load('text8')

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# lets get our NGrams
sentences_tokenized, sentences_bigrams, sentences_trigrams = processSentences(sentences_raw, minStringSize = 5, minTokenCount = 3, splitonPeriod = True, phraseMinCount = 25)


In [4]:
groundTruth = pd.read_csv("data/groundTruth.csv")
groundTruth = groundTruth[['sent','expected']]

def tokenize(sent):
    if type(sent) == str:
        return sent.strip().split(' ')
    else: 
        return []

    
# apply to our expected column
groundTruth['expected'] = groundTruth['expected'].apply(tokenize)

In [5]:
# another way to test this is to do it by set agian...

def testV2(keyedVectors, myVec, groundTruth, threshold):
    
    vocab = set()
    expected = set()
    
    for i in range(len(groundTruth)):
        
        for word in groundTruth['sent'][i].split(' '):
            if word in keyedVectors:
                vocab.add(word)
        
        for word2 in groundTruth['expected'][i]:
            if word2 in keyedVectors:
                expected.add(word2)

    
    # okay so calculate missed:    
    missed = set()
    
    for word in expected:
        if cosineDistance(myVec, keyedVectors[word]) < threshold :
            missed.add(word)
    
    # first need to subtract the expected set.
    notExpected = vocab - expected
    
    over = set()
    
    for word in notExpected:
        if cosineDistance(myVec, keyedVectors[word]) >= threshold :
            over.add(word)
    missedScore = (len(expected)-len(missed))/len(expected)
    overScore = (len(notExpected)-len(over))/len(notExpected)
    
    return [missedScore, overScore, statistics.mean([missedScore, overScore])]

# Now lets just run a big loop

In [6]:
# # okay now we can iterate over our parameters.
# NGRAMS =  [['single', sentences_tokenized], ['bigrams', sentences_bigrams], ['trigrams', sentences_trigrams]]
# WINDOWS = [1,2,3,4,5]
# VECTORSIZE = [100,200,300,400]
# THRESHOLDS = [0.5,0.55,0.6,0.65]

# # results = [ngrams, window, vectorsize, threshold, expected, unexpected, total]
# results = []

# for ngram in NGRAMS:
#     for window in WINDOWS:
#         for vectorSize in VECTORSIZE:
            
#             # train model
#             tempModel = Word2Vec(text8, min_count=25, window=window, sg = 1, vector_size=vectorSize, workers=4)
#             tempModel.build_vocab(ngram[1], update=True)
#             tempModel.train(ngram[1],total_examples=len(ngram[1]), epochs = 5)
#             wv = tempModel.wv
            
#             # find our Vec
#             myVec = meanVector(wv, positive=['gme', 'amc','game_stop','tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'netflix', 'dropbox','slack', 'peloton', 'wal_mart', 'lululemon'], 
#                 negative=[])
            
#             # now we can change the theshold and add store the values:
#             for threshold in THRESHOLDS:
#                 myScores = testV2(wv, myVec, groundTruth, threshold)
#                 results.append([ngram[0], window, vectorSize, threshold, myScores[0], myScores[1], myScores[2]])

                
# finalResults = pd.DataFrame(results, columns = ['ngrams', 'window', 'vectorsize', 'threshold', 'expected', 'unexpected', 'total'])

# finalResults
# finalResults.to_csv('paramAnalysis.csv')

,ngrams,window,vectorsize,threshold,expected,unexpected,total
0,single,1,100,0.50,0.909639,0.875718,0.892678
1,single,1,100,0.55,0.831325,0.943396,0.887361
2,single,1,100,0.60,0.765060,0.971288,0.868174
3,single,1,100,0.65,0.674699,0.982773,0.828736
4,single,1,200,0.50,0.825301,0.933142,0.879222
...,...,...,...,...,...,...,...
235,trigrams,5,300,0.65,0.128655,0.997520,0.563088
236,trigrams,5,400,0.50,0.479532,0.985831,0.732681
237,trigrams,5,400,0.55,0.292398,0.992561,0.642479
238,trigrams,5,400,0.60,0.169591,0.995749,0.582670


# Now lets plot our results.

In [81]:
finalResults.sort_values('total', ascending=False).head(10)

,ngrams,window,vectorsize,threshold,expected,unexpected,total
16,single,2,100,0.50,0.897590,0.894996,0.896
161,trigrams,1,100,0.55,0.859649,0.929153,0.894
0,single,1,100,0.50,0.909639,0.875718,0.893
32,single,3,100,0.50,0.879518,0.904840,0.892
81,bigrams,1,100,0.55,0.847953,0.931627,0.890
1,single,1,100,0.55,0.831325,0.943396,0.887
112,bigrams,3,100,0.50,0.883041,0.888848,0.886
192,trigrams,3,100,0.50,0.883041,0.889834,0.886
96,bigrams,2,100,0.50,0.894737,0.870567,0.883
176,trigrams,2,100,0.50,0.894737,0.871768,0.883


In [45]:
# max = 0.896
# min = 0.5223`
s
# going to round these number so i can label the plots.

finalResults['total'] = finalResults['total'].apply(lambda x: round(x, 3))

# N-GRAM Comparison

In [79]:
TH_50 = finalResults[(finalResults['window'] == 2) & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.5)].reset_index()
TH_55 = finalResults[(finalResults['window'] == 2) & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.55)].reset_index()
TH_60 = finalResults[(finalResults['window'] == 2) & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.6)].reset_index()
TH_65 = finalResults[(finalResults['window'] == 2) & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.65)].reset_index()


TH_60

,index,ngrams,window,vectorsize,threshold,expected,unexpected,total
0,18,single,2,100,0.6,0.698795,0.975390,0.837
1,98,bigrams,2,100,0.6,0.725146,0.970384,0.848
2,178,trigrams,2,100,0.6,0.730994,0.973787,0.852


In [ ]:
# want the min and max to get better axis..



In [80]:
fig = make_subplots(rows=2, cols=2, shared_yaxes=True, subplot_titles=("Threshold = 0.50", "Threshold = 0.55", "Threshold = 0.60", "Threshold = 0.65"))

fig.add_trace(go.Scatter(x=['none', 'bigram', 'trigram'], y=[TH_50['total'][0],TH_50['total'][1],TH_50['total'][2]],
                         text=[TH_50['total'][0],TH_50['total'][1],TH_50['total'][2]],
                         mode="markers+text",
                         textposition="bottom center"),
              row=1, col=1)
fig.add_trace(go.Scatter(x=['none', 'bigram', 'trigram'], y=[TH_55['total'][0],TH_55['total'][1],TH_55['total'][2]],
                         text=[TH_55['total'][0],TH_55['total'][1],TH_55['total'][2]],
                        mode="markers+text",
                        textposition="bottom center"), row=1, col=2)
fig.add_trace(go.Scatter(x=['none', 'bigram', 'trigram'], y=[TH_60['total'][0],TH_60['total'][1],TH_60['total'][2]],
                         text=[TH_60['total'][0],TH_60['total'][1],TH_60['total'][2]],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=1)
fig.add_trace(go.Scatter(x=['none', 'bigram', 'trigram'], y=[TH_65['total'][0],TH_65['total'][1],TH_65['total'][2]],
                         text=[TH_65['total'][0],TH_65['total'][1],TH_65['total'][2]],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=2)

fig.update_yaxes(range=[0.750, 1.0]) 
fig.update_layout(showlegend=False)

fig.update_yaxes(title_text="score", row=1, col=1)
fig.update_yaxes(title_text="score", row=2, col=1)
fig.update_xaxes(title_text="n-gram", row=2, col=1)
fig.update_xaxes(title_text="n-gram", row=2, col=2)

fig.update_layout(
    title="N-Gram Comparison",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)

# Window Comparison

In [61]:
TH_50 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.5)].reset_index()
TH_55 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.55)].reset_index()
TH_60 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.6)].reset_index()
TH_65 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['vectorsize'] == 100) & (finalResults['threshold'] == 0.65)].reset_index()

TH_50

,index,ngrams,window,vectorsize,threshold,expected,unexpected,total
0,80,bigrams,1,100,0.5,0.894737,0.851554,0.873
1,96,bigrams,2,100,0.5,0.894737,0.870567,0.883
2,112,bigrams,3,100,0.5,0.883041,0.888848,0.886
3,128,bigrams,4,100,0.5,0.836257,0.894333,0.865
4,144,bigrams,5,100,0.5,0.789474,0.907130,0.848


In [70]:
fig = make_subplots(rows=2, cols=2, shared_yaxes=True, subplot_titles=("Threshold = 0.50", "Threshold = 0.55", "Threshold = 0.60", "Threshold = 0.65"))

fig.add_trace(go.Scatter(x=TH_50['window'], y=TH_50['total'],
                         text=TH_50['total'],
                         mode="markers+text",
                         textposition="bottom center"),
              row=1, col=1)
fig.add_trace(go.Scatter(x=TH_55['window'], y=TH_55['total'],
                         text=TH_55['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=1, col=2)
fig.add_trace(go.Scatter(x=TH_60['window'],y=TH_60['total'],
                         text=TH_60['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=1)
fig.add_trace(go.Scatter(x=TH_65['window'], y=TH_65['total'],
                        text=TH_65['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=2)

fig.update_yaxes(range=[0.65, 1.0]) 
fig.update_layout(showlegend=False)


fig.update_yaxes(title_text="score", row=1, col=1)
fig.update_yaxes(title_text="score", row=2, col=1)
fig.update_xaxes(title_text="window", row=2, col=1)
fig.update_xaxes(title_text="window", row=2, col=2)

fig.update_layout(
    title="Window Comparison",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)



# Vector Size Comparison

In [72]:
TH_50 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['window'] == 1) & (finalResults['threshold'] == 0.5)].reset_index()
TH_55 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['window'] == 1) & (finalResults['threshold'] == 0.55)].reset_index()
TH_60 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['window'] == 1) & (finalResults['threshold'] == 0.6)].reset_index()
TH_65 = finalResults[(finalResults['ngrams'] == 'bigrams') & (finalResults['window'] == 1) & (finalResults['threshold'] == 0.65)].reset_index()

TH_50

,index,ngrams,window,vectorsize,threshold,expected,unexpected,total
0,80,bigrams,1,100,0.5,0.894737,0.851554,0.873
1,84,bigrams,1,200,0.5,0.818713,0.925777,0.872
2,88,bigrams,1,300,0.5,0.789474,0.937112,0.863
3,92,bigrams,1,400,0.5,0.736842,0.942962,0.840


In [76]:
fig = make_subplots(rows=2, cols=2, shared_yaxes=True, subplot_titles=("Threshold = 0.50", "Threshold = 0.55", "Threshold = 0.60", "Threshold = 0.65"))

fig.add_trace(go.Scatter(x=TH_50['vectorsize'], y=TH_50['total'],
                         text=TH_50['total'],
                         mode="markers+text",
                         textposition="bottom center"),
              row=1, col=1)
fig.add_trace(go.Scatter(x=TH_55['vectorsize'], y=TH_55['total'],
                         text=TH_55['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=1, col=2)
fig.add_trace(go.Scatter(x=TH_60['vectorsize'],y=TH_60['total'],
                         text=TH_60['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=1)
fig.add_trace(go.Scatter(x=TH_65['vectorsize'], y=TH_65['total'],
                        text=TH_65['total'],
                        mode="markers+text",
                        textposition="bottom center"), row=2, col=2)

fig.update_yaxes(range=[0.55, 1.0]) 
fig.update_layout(showlegend=False)


fig.update_yaxes(title_text="score", row=1, col=1)
fig.update_yaxes(title_text="score", row=2, col=1)
fig.update_xaxes(title_text="vector-size", row=2, col=1)
fig.update_xaxes(title_text="vector-size", row=2, col=2)

fig.update_layout(
    title="Vector Size Comparison",
    font=dict(
        family="Courier New, monospace",
        size=14,
    )
)